In [1]:
#Importamos librerias
import logging
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, classification_report
#from sklearn.model_selection import GridSearchCV

#import os
#import matplotlib.pyplot as plt
#import seaborn as sns
#import scipy.stats as stats
#from datetime import datetime
#from sklearn.preprocessing import StandardScaler, OneHotEncoder,MinMaxScaler
#from sklearn.compose import ColumnTransformer
#from sklearn.pipeline import Pipeline
#from sklearn.impute import SimpleImputer
#from sklearn.tree import DecisionTreeClassifier, plot_tree
#from sklearn.metrics import classification_report, confusion_matrix, accuracy_score,roc_curve, auc,precision_recall_curve, average_precision_score
#from sklearn.model_selection import GridSearchCV
#import matplotlib.pyplot as plt

# Configuración básica de log
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    filename='app_AIMLK_M02T03.log',  # Guardar logs en un archivo
    filemode='w'  # Sobreescribe el archivo en cada ejecución
)

def ProcesoAutomatizado():
    logging.info("#Generando semilla\n")
    np.random.seed(42)
    
    logging.info("#Creando datos de ejemplo\n")
    datos = {
        'fecha': pd.date_range(start='2023-12-08', periods=3000, freq='D'),
        'CodCliente': np.random.normal(100000, 500, 3000),
        'Importe': np.random.normal(49400, 500, 3000),
        'Comisión': np.random.normal(494, 150, 3000),
        'Producto': np.random.choice(['Compra BitCoin', 'Cuenta Ahorro', 'Cuenta Corriente', 'DPF'], 3000),
        'Region': np.random.choice(['San Salvador Norte', 'San Salvador Sur', 'San Salvador Este', 'San Salvador Oeste'], 3000),
        'AumentaSaldo': np.random.choice(['Si', 'No'], 3000)
    }
    
    df = pd.DataFrame(datos)
    
    # Simulamos algunos valores nulos y duplicados para el ejemplo
    df.loc[0:5, 'Comisión'] = np.nan
    df = pd.concat([df, df.iloc[0:3]])
    
    logging.info("#Mostrando datos originales:\n")
    logging.info(df)
    
    logging.info('\nInformación de Columnas:\n')
    logging.info(df.info())

    logging.info('# Creamos una copia para no modificar los datos originales')
    df_limpio = df.copy()
    
    logging.info('# 1. Manejo de valores faltantes')
    logging.info('# Rellenamos valores faltantes en columnas numéricas con la mediana')
    columnas_numericas = df_limpio.select_dtypes(include=['float64', 'int64']).columns
    for columna in columnas_numericas:
        if df_limpio[columna].isnull().sum() > 0:
            mediana = df_limpio[columna].median()
            #df_limpio[columna].fillna(mediana, inplace=True)
            df_limpio[columna] = df_limpio[columna].fillna(mediana)
            logging.debug(f"- Valores faltantes en '{columna}' rellenados con la mediana: {mediana}")

    logging.info('# 2. Eliminación de duplicados')
    duplicados = df_limpio.duplicated().sum()
    df_limpio.drop_duplicates(inplace=True)
    logging.debug(f"- Se eliminaron {duplicados} registros duplicados")

    logging.info('# 3. Estandarización de texto')
    columnas_texto = df_limpio.select_dtypes(include=['object']).columns
    for columna in columnas_texto:
        # Convertimos a minúsculas y eliminamos espacios externos
        df_limpio[columna] = df_limpio[columna].str.lower().str.strip()
        logging.debug(f"- Columna '{columna}' estandarizada a minúsculas y sin espacios externos")

    logging.info('# 4. Manejo de valores atípicos (outliers)')
    for columna in columnas_numericas:
        Q1 = df_limpio[columna].quantile(0.25)
        Q3 = df_limpio[columna].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
    
        # Identificamos outliers
        outliers = df_limpio[(df_limpio[columna] < limite_inferior) | 
                            (df_limpio[columna] > limite_superior)][columna]
        
        if len(outliers) > 0:
            # Recortamos los valores atípicos a los límites
            df_limpio[columna] = df_limpio[columna].clip(limite_inferior, limite_superior)
            logging.debug(f"- Se encontraron y trataron {len(outliers)} valores atípicos en '{columna}'")
            
    logging.info('# 5. Creación de nuevas características')
    # Ejemplo: Calcular ratios financieros
    if 'Importe' in df_limpio.columns and 'Comisión' in df_limpio.columns:
        df_limpio['margen_beneficio'] = ((df_limpio['Importe'] - df_limpio['Comisión']) / 
                                        df_limpio['Importe'] * 100)
        logging.debug("- Creada nueva columna 'margen_beneficio'")

    logging.info('# 6. Normalización de datos numéricos')
    for columna in columnas_numericas:
        # Aplicamos normalización Min-Max
        min_val = df_limpio[columna].min()
        max_val = df_limpio[columna].max()
        df_limpio[f'{columna}_normalizado'] = ((df_limpio[columna] - min_val) / 
                                              (max_val - min_val))
        logging.debug(f"- Columna '{columna}' normalizada entre 0 y 1")

    logging.info('# 7. Conversión de tipos de datos')
    logging.info('# Preparando el formato de fecha')
    date_format = '%Y-%m-%d'
    
    logging.info('# Convertir columnas de fecha')
    for columna in columnas_texto:
        try:
            df_limpio[f'{columna}_fecha'] = pd.to_datetime(df_limpio[columna], format=date_format)
            logging.info(f"- Columna '{columna}' convertida a formato fecha")
        except:
            logging.debug(f"- La Columna '{columna}' no es de tipo fecha")
            continue


    logging.info("# 8. Seleccionar columnas necesarias\n")
    final_dataset = df_limpio[['Importe', 'Comisión', 'Producto', 'Region', 'AumentaSaldo']]
    
    
    logging.info("# 9. Separar las características (X) y la variable objetivo (y)")
    logging.info("Distribución de la variable objetivo (AumentaSaldo):")
    logging.info(final_dataset['AumentaSaldo'].value_counts())
    logging.debug(final_dataset['AumentaSaldo'].unique())
    X = final_dataset.drop(columns=['AumentaSaldo'])
    y = final_dataset['AumentaSaldo'].apply(lambda x: 1 if x == 'si' else 0) # Convertir 'si' a 1 y 'no' a 0 para SMOTE
    logging.info(y.value_counts())
    
    logging.info("# 10. Aplicar One-Hot Encoding a las columnas categóricas\n")
    encoder = OneHotEncoder(sparse_output=False, drop='first')  # Cambiamos sparse a sparse_output
    X_encoded = encoder.fit_transform(X[['Producto', 'Region']])
    X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(['Producto', 'Region']))

    logging.info("# 11. Combinar columnas codificadas con las numéricas\n")
    X_numeric = X[['Importe', 'Comisión']].reset_index(drop=True)
    X_final = pd.concat([X_numeric, X_encoded_df], axis=1)

    logging.info("# 12. Dividir datos en entrenamiento y prueba\n")
    X_train, X_test, y_train, y_test = train_test_split(X_final, y, test_size=0.3, random_state=42, stratify=y)

    logging.info("# 13. Aplicar SMOTE para balancear clases\n")
    smote = SMOTE(random_state=42)
    X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

    logging.info("# 14. Distribución después de SMOTE:\n")
    logging.info(y_train_balanced.value_counts())
    
    logging.info('# 15. XGBoost import XGBClassifier:\n')
    logging.debug('n_estimators=500: Número de árboles en el modelo.')
    logging.debug('random_state=42: Fija la semilla para reproducibilidad.')
    logging.debug('max_depth=3:  Profundidad máxima de cada árbol.')
    logging.debug('eval_metric=''logloss'': Métrica de pérdida logarítmica usada internamente durante el entrenamiento.')
    xgb_model = XGBClassifier(n_estimators=500, random_state=42, max_depth=3, eval_metric='logloss')
    xgb_model.fit(X_train_balanced, y_train_balanced)
    
    logging.info('# 16. Predicciones en el conjunto de prueba\n')
    y_pred = xgb_model.predict(X_test)
    
    logging.info(f"# 17. Evaluación del modelo\n")
    logging.info(f"Accuracy del modelo: {accuracy_score(y_test, y_pred)}")
    logging.info(f"\nClassification Report:\n {classification_report(y_test, y_pred)}")

    logging.info(f"# 18. Aplicando Epoch")
    logging.info(f"#     Definir conjuntos de entrenamiento y validación")
    eval_set = [(X_train_balanced, y_train_balanced), (X_test, y_test)]

    logging.info(f"# 19. Entrenar el modelo XGBoost Mejores Parametros")
    #xgb_model = XGBClassifier(n_estimators=500, random_state=42, eval_metric='logloss')
    xgb_model = XGBClassifier(n_estimators=300, learning_rate=0.01, max_depth=12,
                              colsample_bytree=0.8, subsample=0.6,
                             random_state=42, eval_metric='logloss')
    xgb_model.fit(X_train_balanced, y_train_balanced, eval_set=eval_set, verbose=False)

    logging.info(f"# 20. Predicciones")
    y_pred = xgb_model.predict(X_test)
    logging.info(f"\nClassification Report 2:\n {classification_report(y_test, y_pred)}")
    
    logging.info(f"# 21. CONCLUSIONES")
    logging.info(f"Luego de ajustar los parametros del XGBClassifier: ")
    logging.info(f"n_estimators=300 ")
    logging.info(f"learning_rate=0.01 ")
    logging.info(f"max_depth=12 ")
    logging.info(f"colsample_bytree=0.8 ")
    logging.info(f"subsample=0.6 ")
    logging.info(f"random_state=42 ")
    logging.info(f"eval_metric='logloss' ")
    logging.info(f"/n Fue posible mejorar los valores para accuracy, precision y recall!")
    
    
    
   

def main():

    #Titulos 
    logging.info("#-----------------BOF.XGBoots-----------------#")
    logging.info("#######################################################")
    logging.info("# AIML- INCAF-1 Módulo 3: Diseñar e implementar modelos")
    logging.info("## Actividad 1: Construcción de un modelo de Machine Learning utilizando un algoritmo avanzado.")
    logging.info("### Elaborado por: Gabriel Guzmán\n")
    logging.info("Se creado un programa Python el cuál se ejecuta de forma automática las tareas de preparación de datos.")
    logging.info("Se añade el uso de XGBoost para analizar la posibilidad de nuestra variable target si aumentará saldo o no")
    logging.info("\n")
    logging.info("Iniciando Proceso Automatizado...")
    ProcesoAutomatizado()
    logging.info("Fin del Proceso Automatizado.")
    logging.info("#-----------------EOF.XGBoots-----------------#")
    

if __name__ == "__main__":
    main()
    

<class 'pandas.core.frame.DataFrame'>
Index: 3003 entries, 0 to 2
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   fecha         3003 non-null   datetime64[ns]
 1   CodCliente    3003 non-null   float64       
 2   Importe       3003 non-null   float64       
 3   Comisión      2994 non-null   float64       
 4   Producto      3003 non-null   object        
 5   Region        3003 non-null   object        
 6   AumentaSaldo  3003 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 187.7+ KB
